## **2 кейс**

**Выгрузка активности с ItResume**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [211]:
!wget -N https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv

--2025-03-27 12:52:51--  https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv [following]
--2025-03-27 12:52:51--  https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215378 (210K) [text/plain]
Saving to: ‘codesubmit.csv’

codesubmit.csv      100%[===================>] 210.33K  --.-KB/s    in 0.04s   

Last-modified header missing -- time-

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [103]:
import pandas as pd

df = pd.read_csv('codesubmit.csv', sep = ';')
df

,created_at,user_id,problem_id,is_correct,type
0,2023-04-30 13:47:14.344471,7,870,1.0,submit
1,2023-04-30 13:46:15.949925,7,870,0.0,submit
2,2023-04-30 16:13:26.005286,173,21,1.0,submit
3,2023-04-30 16:13:06.739782,173,21,NaN,run
4,2023-04-30 15:52:00.195532,173,25,1.0,submit
...,...,...,...,...,...
4994,2023-04-30 21:52:00.269123,13493,435,NaN,run
4995,2023-04-30 21:51:01.094234,13493,435,1.0,submit
4996,2023-04-30 21:50:52.059690,13493,435,NaN,run
4997,2023-04-30 21:42:24.323689,13493,1086,NaN,run


### **Решения**

#### **Задача 1**

Ваша задача - выяснить сколько в среднем тратится времени на решение задачи.

**Примечание**: для правильного подсчета - рассчитайте сначала среднее время решения по каждой задаче в отдельности, и только затем находите общее среднее время решения задач.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res`.


**Решение**

Напишите свое решение ниже

In [213]:
from datetime import datetime
data = []
with open('codesubmit.csv', 'r') as f:
  data = f.readlines()[1:]
data = [s[:-1].split(';') for s in data]

# Сформировал двумерный массив данных с элементами
data[:5]

[['2023-04-30 13:47:14.344471', '7', '870', '1', 'submit'],
 ['2023-04-30 13:46:15.949925', '7', '870', '0', 'submit'],
 ['2023-04-30 16:13:26.005286', '173', '21', '1', 'submit'],
 ['2023-04-30 16:13:06.739782', '173', '21', '', 'run'],
 ['2023-04-30 15:52:00.195532', '173', '25', '1', 'submit']]

In [214]:
# Заменим первый столбец на тип datetime
for d in data:
  d[0] = datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S.%f')

In [223]:
# Найдём все submit с '1'. Это список всех успешных решений задач
submits = [d[:3] for d in data if d[3] == '1' and d[4] == 'submit']
# Надо найти минимальные дата-время submits для каждой пары (user_id, problem_id)
all = {}
for d in submits:
  key = (d[1], d[2])
  value = (d[0])

  if key not in all or all[key] > value:
    all[key] = value
submits = [[v, k[0], k[1]] for k, v in all.items()]

In [224]:
# Надо найти минимальные дата-время попытки для каждой пары (user_id, problem_id)
all = {}
for d in data:
  key = (d[1], d[2])
  value = (d[0])

  if key not in all or all[key] > value:
    all[key] = value
first_attempts = [[v, k[0], k[1]] for k, v in all.items()]
first_attempts[:5]

[[datetime.datetime(2023, 4, 30, 13, 46, 15, 949925), '7', '870'],
 [datetime.datetime(2023, 4, 30, 16, 13, 6, 739782), '173', '21'],
 [datetime.datetime(2023, 4, 30, 15, 37, 3, 852246), '173', '25'],
 [datetime.datetime(2023, 4, 30, 16, 19, 37, 394470), '173', '35'],
 [datetime.datetime(2023, 4, 30, 15, 14, 13, 931621), '183', '173']]

In [225]:
result = []
for sub in submits:
  for fa in first_attempts:
    if sub[1] == fa[1] and sub[2] == fa[2] and sub[0] != fa[0]:
      # sub[0] != fa[0] отсекает те задачи, которые были решены с первой попытки
        result.append([(sub[0] - fa[0]).total_seconds(), sub[1], sub[2]])
# Теперь в result хранится информация Время выполнения, Пользователь, Задача
result[:5]

[[58.394546, '7', '870'],
 [19.265504, '173', '21'],
 [896.343286, '173', '25'],
 [16560.361198, '173', '35'],
 [4223.552233, '183', '173']]

In [226]:
time = {}
count = {}

for r in result:
  key = r[2]
  value = r[0]
  if key not in time:
    time[key] = value
    count[key] = 1
  else:
    time[key] += value
    count[key] += 1
# В time - Номер задачи: суммарное время её решения
# В count - Номер задачи: количество решений

In [227]:
for key in time:
  time[key] = time[key] / count[key]
# Теперь в time - Номер задачи: среднее время её решения

In [228]:
res = round(sum(time.values())/len(time),2)
res

611.86

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [229]:
try:
    assert res == 611.86
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача - выяснить сколько часов в среднем проводит юзер в день на платформе. Перерывы в активности за день - не учитываем.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res2`.

**Решение**

Напишите свое решение ниже

In [236]:
from datetime import datetime
data = []
with open('codesubmit.csv', 'r') as f:
  data = f.readlines()[1:]
data = [s[:-1].split(';') for s in data]

# Заменим первый столбец на тип datetime
for d in data:
  d[0] = datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S.%f')

In [237]:
# В data второй столбец (d[1]) - id пользователя. Посмотрим когда было его первое касание
all = {}
for d in data:
  key = d[1]
  value = d[0]
  if key not in all or all[key] > value:
    all[key] = value
first_attempts = [[v, k] for k, v in all.items()]

# Аналогично для последнего касания
all = {}
for d in data:
  key = d[1]
  value = d[0]
  if key not in all or all[key] < value:
    all[key] = value
last_attempts = [[v, k] for k, v in all.items()]

In [242]:
time_attempts = []
for fa in first_attempts:
  for la in last_attempts:
    if fa[1] == la[1]:
      time_attempts.append((la[0]-fa[0]).total_seconds()/3600)

In [246]:
res2 = round(sum(time_attempts) / len(time_attempts), 2)
res2

1.7

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [247]:
try:
    assert res2 == 1.7
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Теперь давайте посмотрим на активные сеансы. Выясните, сколько задач в среднем решается за один активный сеанс.

**Активный сеанс** - период, когда между любой активностью пользователя разница менее или равна часу, не более

**Важно**: в расчет берем не только успешные попытки решений (`is_correct=1`), а и неуспешные тоже (`is_correct=0`), и тип `run` в том числе.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res3`.

**Решение**

Напишите свое решение ниже

In [167]:
import csv
from collections import defaultdict
import itertools
import datetime

rows = []
with open('codesubmit.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    for row in csvreader:
        rows.append(row)

In [168]:
from datetime import timedelta

rows.sort(key=lambda x: (x['user_id'], x['created_at']))

dates_by_user = {}
for row in rows:
    user_id = row['user_id']
    problem_id = row['problem_id']
    created_at = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S.%f')
    if user_id not in dates_by_user:
        dates_by_user[user_id] = []
    dates_by_user[user_id].append((created_at, problem_id))

In [170]:
results = {}

for user, submissions in dates_by_user.items():
    sessions = []
    current_session = []

    for i in range(len(submissions)):
        if i == 0:
            current_session.append(submissions[i])
        else:
            time_diff = submissions[i][0] - submissions[i-1][0]
            if time_diff > timedelta(hours=1):
                sessions.append(current_session)
                current_session = [submissions[i]]
            else:
                current_session.append(submissions[i])

    # Добавляем последнюю сессию
    sessions.append(current_session)

    # Считаем количество уникальных решенных задач для каждой сессии
    unique_problems = []
    for session in sessions:
        for submission in session:
            problem_id = submission[1]
            if problem_id not in unique_problems:
                unique_problems.append(problem_id)
        session_count = len(unique_problems)
        unique_problems = []

        # Сохраняем результаты
        if user not in results:
            results[user] = {'sessions': 0, 'problems_solved_per_session': []}

        results[user]['sessions'] += 1
        results[user]['problems_solved_per_session'].append(session_count)

In [171]:
total_sessions = 0
total_problems_solved = 0

for user in results:
    sessions = results[user]['sessions']
    problems_solved = sum(results[user]['problems_solved_per_session'])

    total_sessions += sessions
    total_problems_solved += problems_solved

average_problems_solved_per_session = total_problems_solved / total_sessions

In [172]:
res3 = round(average_problems_solved_per_session, 2)
res3

3.14

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [173]:
try:
    assert res3 == 3.14
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

И финальная - найдите самый "популярный" час дня на нашей платформе.

Популярность определяем максимальным количеством уникальных пользователей, совершающих какую-либо активность в этот период

Результат в числовом формате запишите в переменную `res4`.

Например, самым популярным часом стал период с 22 до 23, тогда в переменной `res4` должно лежать **22**. Обозначающее начало этого периода.

**Решение**

Напишите свое решение ниже

In [83]:
from datetime import datetime
data = []
with open('codesubmit.csv', 'r') as f:
  data = f.readlines()[1:]
data = [s[:-1].split(';') for s in data]

for d in data:
    d[0] = d[0][11:13]

In [101]:
from collections import Counter

value_counts = Counter(d[0] for d in data)

# Находим самое частое значение
most_common_value, most_common_count = value_counts.most_common(1)[0]
res4 = int(most_common_value)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [102]:
try:
    assert res4 == 16
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
